This file is used to construct bilateral tariff data

In [1]:
# Load required libraries
import os
import pandas as pd 
import numpy as np


# Set up

In [2]:
# Set Path
wd = os.path.expanduser("~/Dropbox/Tariff_Project")
os.chdir(wd)
print(f"Current working directory: {os.getcwd()}")

tariff_data_path = os.path.join(wd,"2_Data/Global_Tariff_Database")

trade_share_path = os.path.join(wd, "3_Result/parameters")

list_path = os.path.join(wd, "3_Result/eora_clean")


save_path = os.path.join(wd, "3_Result/parameters")

Current working directory: /Users/lishuangcen/Dropbox/Tariff_Project


# Trade Elasticity

In [3]:
sector_list = (
    pd.read_csv(os.path.join(list_path, "sector_list.csv"), header=0)["0"]
    .astype(str)
    .tolist()
)

print(sector_list)

['Agriculture', 'Fishing', 'Mining and Quarrying', 'Food & Beverages', 'Textiles and Wearing Apparel', 'Wood and Paper', 'Petroleum, Chemical and Non-Metallic Mineral Products', 'Metal Products', 'Electrical and Machinery', 'Transport Equipment', 'Other Manufacturing', 'Recycling', 'Electricity, Gas and Water', 'Construction', 'Maintenance and Repair', 'Wholesale Trade', 'Retail Trade', 'Hotels and Restraurants', 'Transport', 'Post and Telecommunications', 'Finacial Intermediation and Business Activities', 'Public Administration', 'Education, Health and Other Services', 'Private Households', 'Others']


In [34]:
# Trade elasticity: take from "Bolhuis, M. A., Chen, M. J., & Kett, B. R. (2023). Fragmentation in global trade: Accounting for commodities. International Monetary Fund."

# 1 Agriculture: 2.91
# 2 Fishing: 2.91                               
# 3 Mining and Quarrying: 3.41                               
# 4 Food & Beverages: 4.17                                     
# 5 Textiles and Wearing Apparel: 4.71
# 6 Wood and Paper: (8.8 + 8.21) /2 = 8.505                                                                    
# 7 Petroleum, Chemical and Non-Metallic Mineral Products:  (3.67 + 10.56 + 6.75 + 4.79)/4 = 6.4425
# 8 Metal Products: （7.39 + 4.22）/2 = 5.805
# 9 Electrical and Machinery:  (5.01 + 5.14 + 4.11) /3 = 4.753                                                      
# 10 Transport Equipment: (8.92 + 8.99)/2 = 8.955                               
# 11 Other Manufacturing: 4.06
# 12 Recycling: 4.06                
# ---------------------------------------------------------------------
# Services sectors: 8.35

theta = np.array([2.91, 2.91, 3.14, 4.17, 4.71, 8.505, 6.4425, 5.805, 4.753, 8.955, 4.06, 4.06,
                8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35, 8.35])  



sector_names = sector_list          

# Check the length
if len(sector_names) != len(theta):
    raise ValueError(
        f"sector_list has {len(sector_names)} items, "
        f"but θ has length {len(theta)}."
    )

# Construct the maaping
theta_map = dict(zip(sector_names, theta))



# Tariff Data

Sectors' Classification in this tariff dataset follows ISIC Rev3. You can check with the following url

https://unstats.un.org/unsd/classifications/Econ/Structure


In [5]:
# Concordance between 2-digit ISIC Rev 3 and Eora Sector
# Based on Appendix 3 of"Lenzen, M., Moran, D., Kanemoto, K., Geschke, A. (2013)  Building Eora: A Global Multi-regional Input-Output Database at High Country and Sector Resolution. Economic Systems Research, 25:1, 20-49"


# 1.Construct the concordance table
eora_to_isic = {
    "Agriculture":                                                      [1, 2],
    "Fishing":                                                          [5],
    "Mining and Quarrying":                                             [10, 11, 12, 13, 14],
    "Food & Beverages":                                                 [15, 16],
    "Textiles and Wearing Apparel":                                     [17, 18, 19],
    "Wood and Paper":                                                   [20, 21, 22],
    "Petroleum, Chemical and Non-Metallic Mineral Products":            [23, 24, 25, 26],
    "Metal Products":                                                   [27, 28], 
    "Electrical and Machinery":                                         [29, 30, 31, 32, 33],
    "Transport Equipment":                                              [34, 35],
    "Other Manufacturing":                                              [36],
    "Recycling":                                                        [37],
    "Electricity, Gas and Water":                                       [40,41],
    "Construction":                                                     [45],
    "Maintenance and Repair":                                           [50],
    "Wholesale Trade":                                                  [51],
    "Retail Trade":                                                     [52],
    "Hotels and Restraurants":                                          [55],
    "Transport":                                                        [60, 61, 62, 63],
    "Post and Telecommunications":                                      [64], 
    "Finacial Intermediation and Business Activities":                  [65, 66, 67, 70, 71,72, 73, 74],
    "Public Administration":                                            [75],
    "Education, Health and Other Services":                             [80, 85, 90, 91, 92, 93],
    "Private Households":                                               [95],
    "Others":                                                           [99]
}

J = len(eora_to_isic)


In [6]:
# Flattened to a row-level DataFrame (two columns: ISIC, Eora26)
rows = []
for sec_name, codes in eora_to_isic.items():
    for code in codes:
        rows.append({"sector_code": code, "sector_name": sec_name})

concordance = pd.DataFrame(rows)            # sector_code:int, sector_name:str


In [8]:
concordance.head(5)

,sector_code,sector_name
0,1,Agriculture
1,2,Agriculture
2,5,Fishing
3,10,Mining and Quarrying
4,11,Mining and Quarrying


In [9]:
# Import Tariff Data
tariff_df = pd.read_csv(os.path.join(tariff_data_path,"tariff_isic33_88_21_vbeta1-2024-12.csv"))

# for sector in tariff_df["sector"].unique():
#     print(sector)

# tariff_df["year"].unique()

In [10]:
# Select the year and sector we need
# Variable notes：iso1: Importer; iso2: Exporter
# tariff: Unweighted average bilateral tariff
# mfn: Unweighted average MFN tariff, varies by i (or i–sector, if applicable)
# tariff95: Unweighted average bilateral tariff with winsorized outliers
# mfn95: Unweighted average MFN tariff with winsorized outliers

tariff_2017 = tariff_df[tariff_df["year"] == 2017].copy()


In [11]:
# Check if "tariff", "mfn", "tariff95" and "mfn95" have NA, seperately

tariff_col = ["tariff", "mfn", "tariff95", "mfn95"]  

# 1. Count NA
na_counts = tariff_2017[tariff_col].isna().sum()
print("NA counts per column:")
print(na_counts)

# 2. Check if NA exist
has_na = tariff_2017[tariff_col].isna().any()
print("\nDoes the column contain NA?")
print(has_na)


NA counts per column:
tariff      0
mfn         0
tariff95    0
mfn95       0
dtype: int64

Does the column contain NA?
tariff      False
mfn         False
tariff95    False
mfn95       False
dtype: bool


In [12]:
# Keep columns we need: here, I temporirally keep all 4 kinds of tariff, and see if they make significant differences in the estimation

keep_cols = ["iso1", "iso2", "sector", "tariff", "mfn", "tariff95", "mfn95"]

tariff_2017_trim = tariff_2017[keep_cols].copy()


In [13]:
# Select the sectors we need
allowed_isic = [code                
                for codes in eora_to_isic.values()
                for code  in codes]


tariff_2017_subsector = (
    tariff_2017_trim[tariff_2017_trim["sector"].astype(int).isin(allowed_isic)]
    .copy()
)

# tariff_2017_subsector.head(60)

In [14]:
# Check if the Tariff data constians all country-pair tariffs
# Variable notes：iso1: Importer; iso2: Exporter

# 0. Import country list
country_list = (
    pd.read_csv(os.path.join(list_path, "country_list.csv"), header=0)["0"]
    .astype(str)
    .tolist()
)

N = len(country_list)
# exclude ROW
core_countries = country_list[:-1] if country_list[-1] == "ROW" else country_list

col_o = "iso1"    # importer
col_d = "iso2"    # exporter

# 1. construct all possible country pairs 
expected_pairs = {(i, j) for i in core_countries for j in core_countries if i != j}

# 2. country pairs existing in Tariff dataset
observed_pairs = set(zip(tariff_2017_subsector[col_o], tariff_2017_subsector[col_d]))

# 3. Check
missing_pairs = expected_pairs - observed_pairs

if not missing_pairs:
    print("✅ All Exist")
else:
    print(f"⚠️ {len(missing_pairs)} pairs absent, for exmaple (first 10)：")
    for p in list(missing_pairs)[:10]:
        print("   ", p)


✅ All Exist


In [16]:
# Construct ROW Tariff

# 1. label countries not in ccore_countries as ROW
tariff_constructed = tariff_2017_subsector.copy()   

for col in ["iso1", "iso2"]:
    tariff_constructed[col] = tariff_constructed[col].where(tariff_constructed[col].isin(core_countries), "ROW")

# 2. delete ROW-ROW rows
tariff_constructed = tariff_constructed[~((tariff_constructed["iso1"] == "ROW") & (tariff_constructed["iso2"] == "ROW"))]

# 3. Divide the data to "core-core" and "country-ROW"
row_side_mask = (tariff_constructed["iso1"] == "ROW") | (tariff_constructed["iso2"] == "ROW")
df_row   = tariff_constructed.loc[row_side_mask].copy()       # Take average then
df_core  = tariff_constructed.loc[~row_side_mask].copy()      # keep as it is 


# 4.group by iso1, iso2, sector and take average
df_row_avg = (
    df_row
    .groupby(["iso1", "iso2", "sector"], as_index=False)[tariff_col]
    .mean()
)

# 5. merge back to df_core
tariff_constructed_final = pd.concat([df_core, df_row_avg], ignore_index=True)

print(f"# of rows: {len(tariff_constructed_final):,}")
print(tariff_constructed_final.head())

# of rows: 46,620
  iso1 iso2  sector    tariff       mfn  tariff95     mfn95
0  AUS  AUT       1  0.396300  0.396300  0.396300  0.396300
1  AUS  AUT       2  0.416667  0.416667  0.416667  0.416667
2  AUS  AUT       5  0.060976  0.060976  0.060976  0.060976
3  AUS  AUT      10  0.000000  0.000000  0.000000  0.000000
4  AUS  AUT      11  0.000000  0.000000  0.000000  0.000000


In [20]:
# Concordance to Eora Sector 
# Take average across sectors

# 0. Concordance to Eora Sector
# concordance has been defined before

# 2. Merge: numeric sector → string department name

df = tariff_constructed_final.copy()

# check the class of the data
df["sector_code"] = df["sector"].astype(int)

df = (
    df.merge(concordance, on="sector_code", how="left", validate="m:1")
    .drop(columns=["sector", "sector_code"])       
    .rename(columns={"sector_name": "sector"})
)

# 3. Take average

tariff_eora_avg = (
    df
    .groupby(["iso1", "iso2", "sector"], as_index=False)[tariff_col]
    .mean()
)

# Check if the tariff data contains all trade combination
expected_rows = (N - 1) * N * J
actual_rows   = len(tariff_eora_avg)

print(f"Expected rows = (N-1)*N*J_trade = ({N-1})*{N}*{J} = {expected_rows:,}")
print(f"Actual rows   = {actual_rows:,}")

if actual_rows == expected_rows:
    print("✅ The number of rows matches！")
else:
    print("⚠️ The number of rows not match, run the following cell to check the lost rows")


# rename the columns
tariff_eora_avg.rename(columns={
    "iso1":    "Importer",
    "iso2":    "Exporter",
    "sector":  "Sector"
}, inplace=True)



Expected rows = (N-1)*N*J_trade = (36)*37*25 = 33,300
Actual rows   = 19,980
⚠️ The number of rows not match, run the following cell to check the lost rows


In [21]:
tariff_eora_avg.head()

,Importer,Exporter,Sector,tariff,mfn,tariff95,mfn95
0,AUS,AUT,Agriculture,0.406483,0.406483,0.406483,0.406483
1,AUS,AUT,"Education, Health and Other Services",0.000000,0.000000,0.000000,0.000000
2,AUS,AUT,Electrical and Machinery,1.777549,1.777549,1.777549,1.777549
3,AUS,AUT,"Electricity, Gas and Water",1.666667,1.666667,1.666667,1.666667
4,AUS,AUT,Finacial Intermediation and Business Activities,1.666000,1.666000,1.666000,1.666000


In [24]:
# The row-count mismatch tells us that some (Importer, Exporter, Sector) combinations
# are missing from our current tariff dataframe.

# To make analysis convinient, first build a *balanced* tariff table that
# explicitly includes every possible (Importer, Exporter, Sector) combinations
# For combinations that don’t exist in the original data, add a row and set the tariff columns to 0

import itertools
# ------------------------------------------------------------
# 1. Generate the complete Importer × Exporter × Sector grid  
#    Order: Importer (outer loop) → Exporter → Sector — matches your example
# ------------------------------------------------------------

full_tariff = pd.DataFrame(
    list(itertools.product(country_list, country_list, sector_list)),
    columns=["Importer", "Exporter", "Sector"]
)

# ------------------------------------------------------------
# 2. Left-join the existing tariff data onto this grid  
#    Missing combinations will appear as NaN
# ------------------------------------------------------------
full_tariff = full_tariff.merge(
    tariff_eora_avg,
    on=["Importer", "Exporter", "Sector"],
    how="left",
    sort=False                 # don't let 'merge' change the order (important)
)
# ------------------------------------------------------------
# 3. Replace all missing tariff values with 0
# ------------------------------------------------------------
full_tariff[tariff_col] = full_tariff[tariff_col].fillna(0)



In [42]:
# This is the end of tariff data manipulation, the final result is as 
full_tariff.head(30)

full_tariff.to_csv(os.path.join(save_path, "All_Tariff_2017.csv"))

# Trade Cost Calculation

In [28]:
# Import trade share data

pi_intermediate = pd.read_csv(os.path.join(trade_share_path, "trade_share_inter_2017.csv"))

pi_final = pd.read_csv(os.path.join(trade_share_path, "trade_share_final_2017.csv"))


In [ ]:
# Check if trade share data is correct (sum of share should be equal to 1)

# -------------------------------
# 1. group-by and sum the shares
# -------------------------------
pi_df = pi_final.copy()

check = (
    pi_df
    .groupby(["Importer", "Sector"], as_index=False)["Share"]
    .sum()
    .rename(columns={"Share": "sum_share"})
)

# -------------------------------
# 2. flag rows that deviate
# -------------------------------
tol = 1e-6                      # tolerance for rounding / float noise
problem = check.loc[~np.isclose(check["sum_share"], 1.0, atol=tol)]

if problem.empty:
    print("✅ All (Importer, Sector) groups sum to 1.")
else:
    print(f"⚠️ {len(problem)} groups do NOT sum to 1 (|sum-1| > {tol}).")
    display(problem.head())     # show first few offending rows


✅ All (Importer, Sector) groups sum to 1.


In [40]:
from Functions import cal_trade_cost

# final goods using tariff
d_final_tariff = cal_trade_cost(pi_final, full_tariff, theta_map, tariff_col="tariff")
d_final_tariff.to_csv(os.path.join(save_path, "d_final_tariff.csv"))

# intermediate goods using tariff
d_inter_tariff = cal_trade_cost(pi_intermediate, full_tariff, theta_map, tariff_col="tariff")
d_inter_tariff.to_csv(os.path.join(save_path, "d_inter_tariff.csv"))



In [38]:
tariff_eora_avg.head()

,Importer,Exporter,Sector,tariff,mfn,tariff95,mfn95
0,AUS,AUT,Agriculture,0.406483,0.406483,0.406483,0.406483
1,AUS,AUT,"Education, Health and Other Services",0.000000,0.000000,0.000000,0.000000
2,AUS,AUT,Electrical and Machinery,1.777549,1.777549,1.777549,1.777549
3,AUS,AUT,"Electricity, Gas and Water",1.666667,1.666667,1.666667,1.666667
4,AUS,AUT,Finacial Intermediation and Business Activities,1.666000,1.666000,1.666000,1.666000


In [39]:
full_tariff.head()

,Importer,Exporter,Sector,tariff,mfn,tariff95,mfn95
0,AUS,AUS,Agriculture,0.0,0.0,0.0,0.0
1,AUS,AUS,Fishing,0.0,0.0,0.0,0.0
2,AUS,AUS,Mining and Quarrying,0.0,0.0,0.0,0.0
3,AUS,AUS,Food & Beverages,0.0,0.0,0.0,0.0
4,AUS,AUS,Textiles and Wearing Apparel,0.0,0.0,0.0,0.0
